# Enregistrement de tables de variances expliquées

Ce notebook permet d'enregistrer les tables de variances expliquées pour streamlite.

In [1]:
from pickleshare import PickleShareDB
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Chargement des données

Nous chargeons les données depuis le fichier des données préparées.

In [2]:
db = PickleShareDB('../prep_data/kity')

In [3]:
# Données nettoyées
df_net_1 = db['net_attack_1_clean']
df_net_2 = db['net_attack_2_clean']
df_net_3 = db['net_attack_3_clean']
df_net_4 = db['net_attack_4_clean']
df_net_norm = db['net_norm_clean']

In [4]:
# Création d'un dataset avec toutes les données
#df = [db['net_attack_1_clean'], db['net_attack_2_clean'], db['net_attack_3_clean'], db['net_attack_4_clean'], db['net_norm_clean']]
#df = pd.concat(df, axis=0, ignore_index=True)

## Préparation des données

In [5]:
def preparation(df):
    df = df.drop(columns=['Time'])
    scaler = StandardScaler()
    df[['n_pkt_src', 'n_pkt_dst', 'size']] = scaler.fit_transform(df[['n_pkt_src', 'n_pkt_dst', 'size']])
    labels = df[['label', 'label_n']]
    df = df.drop(columns=['label', 'label_n'])
    df = pd.get_dummies(df, columns=['flags',  'proto', 'modbus_fn', 'modbus_response', 'dport', 'sport'], prefix=['flags', 'proto', 'modbus_fn', 'modbus_response', 'dport', 'sport'])
    ordinal_encoder = OrdinalEncoder()
    df[['ip_s', 'ip_d', 'mac_s', 'mac_d']] = ordinal_encoder.fit_transform(df[['ip_s', 'ip_d', 'mac_s', 'mac_d']])
    scaler = StandardScaler()
    df[['ip_s', 'ip_d', 'mac_s', 'mac_d']] = scaler.fit_transform(df[['ip_s', 'ip_d', 'mac_s', 'mac_d']])
    df.drop(columns=['modbus_fn_inconnue', 'flags_inconnue',  'modbus_response_inconnue'], inplace=True) #'sport_inconnu', 'dport_inconnu',
    for col in df.columns:
        if df[col].dtype == 'bool':
            df[col] = df[col].astype(float)
    return df, labels

In [6]:
df_net_1, labels_1 =  preparation(df_net_1)
print("df_net_1 prêt")
df_net_2, labels_2 =  preparation(df_net_2)
print("df_net_2 prêt")
df_net_3, labels_3 =  preparation(df_net_3)
print("df_net_3 prêt")
df_net_4, labels_4 =  preparation(df_net_4)
print("df_net_4 prêt")
df_net_norm, labels_norm =  preparation(df_net_norm)
#print("df_net_norm prêt")
#df, labels = preparation(df)

df_net_1 prêt
df_net_2 prêt
df_net_3 prêt
df_net_4 prêt


## Réduction de dimensions : PCA

### Variance expliquée par les dimensions

In [7]:
def pca(df, name):
    pca = PCA()
    df_pca = pca.fit(df)
    variance_expliquee = (pca.explained_variance_ratio_ * 100).round(2)
    #variance_expliquee = variance_expliquee.round(2)

    variance_table = pd.DataFrame({
        "Composante": [f"PC{i+1}" for i in range(len(variance_expliquee))],
        "Pourcentage de Variance Expliquée": variance_expliquee,
        "Total": variance_expliquee.cumsum()
    })

    db['pca_variance_table_'+ name] = variance_table

In [8]:
#pca(df, 'all')
pca(df_net_1, 'net_1')
pca(df_net_2, 'net_2')
pca(df_net_3, 'net_3')
pca(df_net_4, 'net_4')
pca(df_net_norm, 'net_norm')

Le nombre de dimensions minimum permettant de conserver au moins 90% de l'information :
- df_1 : 6 dimensions (PC6  2.65  91.94)
- df_2 : 6 dimensions (PC6  2.26  91.29)
- df_3 : 4 dimensions (PC4  2.33  91.95)
- df_4 : 6 dimensions (PC6  2.62  90.53)
- df_norm : pas d'interêt
- all : 6 dimensions (PC6  2.22  91.00)